In [7]:
from lib.dataloader import get_MNIST_loaders

import os
import torch

In [8]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
path = os.path.dirname(os.path.abspath("__file__"))
data_path = path + "\\data"

In [6]:
batch_size=128

## Get data loaders

In [9]:
train_loader, val_loader, test_loader = get_MNIST_loaders(data_path, range(10), batch_size)